# Bài Tập
1. Từ code demo hãy cài đặt thêm một module để chọn ra được bộ weights sao cho accuracy trên tập validation là tốt nhất.
2. Từ bộ dữ liệu bên dưới hãy cài đặt backpropagation cho bài toán phân biệt ung thư vú. Hãy tự chọn số layers và số nodes mà mình cho là thích hợp, cũng như là nêu ra số layers và số nodes của mỗi layer mà mình đã chọn. Tính accuracy trên tập training.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

## Câu 1

In [2]:
# Load data
train = pd.read_csv('bt_train.csv')
valid = pd.read_csv('bt_valid.csv')

In [3]:
# Preprocessing
def one_hot_vector(y):
    out = np.zeros((y.shape[0], max(y) + 1))
    for i in range(y.shape[0]):
        out[i, y[i]] = 1
    return out

In [4]:
x1_train = train['x1'].values
x2_train = train['x2'].values
y_train = train['label'].values
x1_valid = valid['x1'].values
x2_valid = valid['x2'].values
y_valid = valid['label'].values
# Normalize
x1_mean = np.mean(x1_train)
x1_std = np.std(x1_train)
x2_mean = np.mean(x2_train)
x2_std = np.std(x2_train)
x1_train = (x1_train - x1_mean) / x1_std
x2_train = (x2_train - x2_mean) / x2_std
x1_valid = (x1_valid - x1_mean) / x1_std
x2_valid = (x2_valid - x2_mean) / x2_std
X_train = np.concatenate([x1_train.reshape(-1, 1), x2_train.reshape(-1, 1)], axis=1)
y_train = one_hot_vector(y_train)
X_valid = np.concatenate([x1_valid.reshape(-1, 1), x2_valid.reshape(-1, 1)], axis=1)

In [5]:
# Initialize
np.random.seed(4)
W1 = np.random.randn(2, 5)
W2 = np.random.randn(5, 5)
W3 = np.random.randn(5, 3)
W4 = np.random.randn(3, 3)
b1 = np.random.randn(1, 5)
b2 = np.random.randn(1, 5)
b3 = np.random.randn(1, 3)
b4 = np.random.randn(1, 3)

In [6]:
def relu(h):
    return np.array([max(0, i) for i in h.reshape(-1)]).reshape(h.shape)


def softmax(z):
    return np.exp(z) / np.sum(np.exp(z), axis=1).reshape(-1, 1)


ln = 0.052
N = y_train.shape[0]

In [7]:
for epochs in range(30):
    # foward
    z1 = np.dot(X_train, W1) + b1
    o1 = relu(z1)
    z2 = np.dot(o1, W2) + b2
    o2 = relu(z2)
    z3 = np.dot(o2, W3) + b3
    o3 = np.tanh(z3)
    z4 = np.dot(o3, W4) + b4
    o4 = softmax(z4)
    # backpropagation
    dL_dz4 = 1 / len(X_train) * (o4 - y_train)
    dL_dW4 = np.dot(o3.T, dL_dz4)
    dL_db4 = np.sum(dL_dz4, axis=0)
    dL_do3 = np.dot(dL_dz4, W4.T)
    dL_dz3 = dL_do3.copy()
    dL_dz3[z3 < 0] = 0
    dL_dW3 = np.dot(o2.T, dL_dz3)
    dL_db3 = np.sum(dL_dz3, axis=0)
    dL_do2 = np.dot(dL_dz3, W3.T)
    dL_dz2 = dL_do2.copy()
    dL_dz2[z2 < 0] = 0
    dL_dW2 = np.dot(o1.T, dL_dz2)
    dL_db2 = np.sum(dL_dz2, axis=0)
    dL_do1 = np.dot(dL_dz2, W2.T)
    dL_dz1 = dL_do1.copy()
    dL_dz1[z1 < 0] = 0
    dL_dW1 = np.dot(X_train.T, dL_dz1)
    dL_db1 = np.sum(dL_dz1, axis=0)
    W4 -= ln * dL_dW4
    b4 -= ln * dL_db4
    W3 -= ln * dL_dW3
    b3 -= ln * dL_db3
    W2 -= ln * dL_dW2
    b2 -= ln * dL_db2
    W1 -= ln * dL_dW1
    b1 -= ln * dL_db1

In [8]:
z1_valid = np.dot(X_valid, W1) + b1
o1_valid = relu(z1_valid)
z2_valid = np.dot(o1_valid, W2) + b2
o2_valid = relu(z2_valid)
z3_valid = np.dot(o2_valid, W3) + b3
o3_valid = np.tanh(z3_valid)
z4_valid = np.dot(o3_valid, W4) + b4
o4_valid = softmax(z4_valid)

In [9]:
np.sum(np.argmax(o4_valid, axis=1) == y_valid) / y_valid.shape[0]

0.7

## Câu 2

In [10]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_mean = np.mean(X_train)
X_std = np.std(X_train)
X_valid = (X_valid - X_mean) / X_std
X_train = (X_train - X_mean) / X_std

In [11]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s


def layers(X, Y):
    n_x = X.shape[0]
    n_y = Y.shape[0]
    return n_x, n_y


def initialize(n_x, n_h, n_y):
    np.random.seed(2)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.random.rand(n_h, 1)
    W2 = np.random.rand(n_y, n_h)
    b2 = np.random.rand(n_y, 1)
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters


def forward_prop(X, parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    return A2, cache


def compute_cost(A2, Y, parameters):
    m = Y.shape[1]
    W1 = parameters['W1']
    W2 = parameters['W2']
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1 - A2), (1 - Y))
    cost = - np.sum(logprobs) / m
    cost = np.squeeze(cost)
    return cost


def back_prop(parameters, cache, X, Y):
    m = Y.shape[1]
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
    dZ2 = A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.square(A1))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    return grads


def update_params(parameters, grads, alpha):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters


def model(X, Y, X_valid, y_valid, n_h, num_iters, alpha):
    np.random.seed(3)
    n_x = layers(X, Y)[0]
    n_y = layers(X, Y)[1]
    parameters = initialize(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    costs = []
    for i in range(0, num_iters):
        A2, cache = forward_prop(X, parameters)
        cost = compute_cost(A2, Y, parameters)
        grads = back_prop(parameters, cache, X, Y)
        if (i > 20000):
            alpha1 = (20000 / i) * alpha
            parameters = update_params(parameters, grads, alpha1)
        else:
            parameters = update_params(parameters, grads, alpha)

    predictions = predict(parameters, X)
    truePositive = 0
    trueNegative = 0
    falseNegative = 0
    falsePositive = 0
    predList = predictions.tolist()
    tlist = Y.tolist()
    array_length = len(predList[0])
    for i in range(array_length):
        if predList[0][i] == 1 and tlist[0][i] == 1:
            truePositive += 1
        elif predList[0][i] == 0 and tlist[0][i] == 0:
            trueNegative += 1
        elif predList[0][i] == 0 and tlist[0][i] == 1:
            falseNegative += 1
        elif predList[0][i] == 1 and tlist[0][i] == 0:
            falsePositive += 1
        else:
            print(predList[0][i])
            print(tlist[0][i])
            print("WTF")
    tpr = truePositive / (truePositive + falseNegative) * 100
    fpr = falsePositive / (falsePositive + trueNegative) * 100
    precision = truePositive / (truePositive + falsePositive) * 100
    print("True Positive Rate / Recall: %.2f" % tpr + str('%'))
    print("Precision: %.2f" % precision + str('%'))
    print("False Positive Rate / Fallout: %.2f" % fpr + str('%'))
    return parameters


def predict(parameters, X):
    A2, cache = forward_prop(X, parameters)
    predictions = np.round(A2)
    return predictions

In [12]:
y_train = one_hot_vector(y_train)
y_valid = one_hot_vector(y_valid)
d = model(X_train.T, y_train.T, X_valid.T, y_valid.T, n_h=20, num_iters=50000, alpha=0.0002)

True Positive Rate / Recall: 81.66%
Precision: 94.52%
False Positive Rate / Fallout: 2.80%
